In [1]:
import stanza.runtime
stanza.runtime.setup()

from stanza.data import PyTreeData
from stanza import train

from typing import Sequence

import optax
import jax
import jax.numpy as jnp
import flax.linen as nn
import stanza.train.ipython
import stanza.train.wandb

In [2]:
X = jnp.arange(1000).reshape((100, 10))
Y = (jnp.arange(100) > 50).reshape((100,))
train_data = PyTreeData((X, Y))

In [3]:
class SimpleMLP(nn.Module):
    features: Sequence[int]

    @nn.compact
    def __call__(self, inputs):
        x = inputs
        for i, feat in enumerate(self.features):
            x = nn.Dense(feat, name=f'layers_{i}')(x)
            if i != len(self.features) - 1:
                x = nn.relu(x)
        return x

def loss(vars, rng_key, sample, iteration):
    x, y = sample
    y_hat = model.apply(vars, x)
    loss = jnp.sum(jnp.square(y_hat - y))
    return train.LossOutput(
        loss=loss,
        metrics={"loss": loss}
    )
loss_fn = train.batch_loss(loss)

model = SimpleMLP([20, 10])

In [5]:
import wandb
run = wandb.init(reinit=True)
print(run.url)

optimizer = optax.adamw(1e-4)

vars = model.init(jax.random.key(42), jnp.zeros_like(train_data.structure[0]))
opt_state = optimizer.init(vars["params"])

with train.loop(train_data, 
            batch_size=16, 
            rng_key=jax.random.key(42),
            iterations=1000,
            progress=True
        ) as loop:
    for epoch in loop.epochs():
        for step in epoch.steps():
            # *note*: consumes opt_state, vars
            opt_state, vars, metrics = train.step(
                loss_fn, optimizer, opt_state, vars, 
                step.rng_key, step.batch,
                # extra arguments for the loss function
                iteration=step.iteration
            )
            if step.iteration % 100 == 0:
                train.ipython.log(step.iteration, metrics)
                train.wandb.log(step.iteration, metrics, run=run)
    train.ipython.log(step.iteration, metrics)
    train.wandb.log(step.iteration, metrics, run=run)

https://wandb.ai/dpfrommer-projects/stanza-projects_examples_notebooks_common/runs/wm25rwnw


Iteration       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1000/1000 100% 0:00:00 0:00:01
Epoch           ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167/167   100% 0:00:00 0:00:01
Epoch Iteration ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4       100% 0:00:00 0:00:00


[15:56:55] INFO     stanza.train -      0 | loss: 1134275.625
           INFO     stanza.train -    100 | loss: 558311.9375
           INFO     stanza.train -    200 | loss: 375918.28125
           INFO     stanza.train -    300 | loss: 123352.2109375
[15:56:56] INFO     stanza.train -    400 | loss: 133825.625
           INFO     stanza.train -    500 | loss: 56262.05859375
           INFO     stanza.train -    600 | loss: 37692.49609375
           INFO     stanza.train -    700 | loss: 8036.95751953125
           INFO     stanza.train -    800 | loss: 3596.23828125
           INFO     stanza.train -    900 | loss: 4816.80078125
           INFO     stanza.train -    999 | loss: 1787.5040283203125
